In [1]:
import pandas as pd
import numpy as np
from pandas.io import json
import requests


Play with some basic functions adapted from [tide data functions](https://github.com/baumanab/seattle_tides)

### Query Builder

In [1]:
def query_builder(start_dt, end_dt, station, offset= 1):

    """Function accepts: a start and end datetime string in the form 'YYYYMMDD mm:ss'
    which are <= 1 year apart, a station ID, and an offset. 
    Function assembles a query parameters/arguments dict and returns an API query and the 
    query dictionary (query_dict). The relevant base URL is the NCDC endpoint 
    'http://www.ncdc.noaa.gov/cdo-web/api/v2/data?'."""

    import urllib
    
    # API endpoint
    base_url= 'http://www.ncdc.noaa.gov/cdo-web/api/v2/data?'

    # dict of NOAA query parameters/arguments

    query_dict = dict(startdate= start_dt, enddate= end_dt, stationid= station,
                      offset= offset, datasetid= 'GHCND', limit= 1000)

    # encode arguments

    encoded_args = urllib.urlencode(query_dict)
    
    # query
    query = base_url + encoded_args
    
    # decode url % (reconvert reserved characters to utf8 string)
    query= urllib.unquote(query)

    # create and return query from base url and encoded arguments
    return query, query_dict

In [5]:
query_1, query_dict= query_builder('2014-01-01', '2015-01-01', station= 'GHCND:USW00023174')
print(query_1)

http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-01-01&stationid=GHCND:USW00023174&enddate=2015-01-01&offset=1&limit=1000&datasetid=GHCND


In [6]:
query_2, query_dict= query_builder('2014-01-01', '2015-01-01', station= 'GHCND:USW00023174', offset= 1001)
print(query_2)

http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-01-01&stationid=GHCND:USW00023174&enddate=2015-01-01&offset=1001&limit=1000&datasetid=GHCND


### Offset Generator

In [7]:
def offsetter(response):
    
    # get repeats and repeat range
    import math
    count= response['metadata']['resultset']['count']
    repeats= math.ceil(count/1000.)
    repeat_range= range(int(repeats))
    
    # get offsets dictionary
    
    offset= 1
    offsets= [1]
    for item in repeat_range[1:]:
        offset += 1000
        offsets.append(offset)
        
    
    # zip up the results and convert to dictionary
    offset_dict= dict(zip(repeat_range[1:], offsets[1:])) # the first call has been done already to get meta
    
    return offset_dict, count # for quality control 
    
    

### Query Generator

#### TODO
- refactor with a decorator
- make key an attribute that can be hidden

In [8]:
def execute_query(query):
    url = query
    # replace token with token provided by NOAA.  Enter token as string
    headers = {'token': NOAA_Token_Here}
    response = requests.get(url, headers = headers)
    response = response.json()
    
    return response

In [46]:
working_1= execute_query(query_1)['results']
working_2 = execute_query(query_2)['results']

In [73]:
check= yeah.pivot(index= 'date',columns= 'datatype', values= 'value').reset_index()

In [74]:
check[check.date.isnull()]

datatype,date,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT08


### Extract Results

In [75]:
def extract_results(response):
    data= response['results']
    # for quality control to verify retrieval of all rows
    length= len(data)
    
    return data, length

In [85]:
def collator(results):
    
    df= pd.DataFrame(results)    
    df= df.drop(['attributes','station'], axis=1)
    df= df.pivot(index= 'date',columns= 'datatype', values= 'value').reset_index()
    
    return df

In [102]:
def get_ncdc(start_dt, end_dt, station):
    
    
    # count for verifying retrieval of all rows
    record_count= 0
    # initial query
    query, query_dict= query_builder(start_dt, end_dt, station)
    response= execute_query(query)
    
    #  extract results and count    
    results, length= extract_results(response)
    record_count += length
    
    # get offsets for remaining queries
    off_d, count= offsetter(response)
    
    # execute remaining queries and operations
    for offset in off_d:
        query, _= query_builder(start_dt, end_dt, station, off_d[offset])
        print(query)
        response= execute_query(query)
        next_results, next_length= extract_results(response)
        
        record_count += next_length
        
        # concat results lists
        results += next_results
        
    assert record_count == count, 'record count != count'
    
    collated_data= collator(results)
        
    return collated_data, record_count
        
    

In [103]:
test, qc = get_ncdc('2014-01-01', '2014-12-31', station= 'GHCND:USW00023174')

http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-01-01&stationid=GHCND:USW00023174&enddate=2014-12-31&offset=1001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-01-01&stationid=GHCND:USW00023174&enddate=2014-12-31&offset=2001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-01-01&stationid=GHCND:USW00023174&enddate=2014-12-31&offset=3001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-01-01&stationid=GHCND:USW00023174&enddate=2014-12-31&offset=4001&limit=1000&datasetid=GHCND


In [104]:
test.date.head()

0    2014-01-01T00:00:00
1    2014-01-02T00:00:00
2    2014-01-03T00:00:00
3    2014-01-04T00:00:00
4    2014-01-05T00:00:00
Name: date, dtype: object

In [105]:
test.date.tail()

360    2014-12-27T00:00:00
361    2014-12-28T00:00:00
362    2014-12-29T00:00:00
363    2014-12-30T00:00:00
364    2014-12-31T00:00:00
Name: date, dtype: object

In [106]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 16 columns):
date    365 non-null object
AWND    365 non-null float64
PRCP    365 non-null float64
SNOW    357 non-null float64
SNWD    360 non-null float64
TAVG    365 non-null float64
TMAX    365 non-null float64
TMIN    365 non-null float64
WDF2    365 non-null float64
WDF5    354 non-null float64
WSF2    365 non-null float64
WSF5    354 non-null float64
WT01    94 non-null float64
WT02    5 non-null float64
WT03    1 non-null float64
WT08    37 non-null float64
dtypes: float64(15), object(1)
memory usage: 44.2+ KB


In [107]:
test[test.date.isnull()]

datatype,date,AWND,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT08


In [111]:
y1, qc = get_ncdc('2014-05-03', '2015-05-02', station= 'GHCND:USW00023174')
y2, qc = get_ncdc('2015-05-03', '2016-05-02', station= 'GHCND:USW00023174')
y3, qc = get_ncdc('2016-05-03', '2017-05-02', station= 'GHCND:USW00023174')

http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-05-03&stationid=GHCND:USW00023174&enddate=2015-05-02&offset=1001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-05-03&stationid=GHCND:USW00023174&enddate=2015-05-02&offset=2001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-05-03&stationid=GHCND:USW00023174&enddate=2015-05-02&offset=3001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2014-05-03&stationid=GHCND:USW00023174&enddate=2015-05-02&offset=4001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2015-05-03&stationid=GHCND:USW00023174&enddate=2016-05-02&offset=1001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2015-05-03&stationid=GHCND:USW00023174&enddate=2016-05-02&offset=2001&limit=1000&datasetid=GHCND
http://www.ncdc.noaa.gov/cdo-web/api/v2/data?startdate=2015-05-03&stationid=GHCND:USW000

In [112]:
y1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 16 columns):
date    365 non-null object
AWND    365 non-null float64
PRCP    365 non-null float64
SNOW    358 non-null float64
SNWD    360 non-null float64
TAVG    365 non-null float64
TMAX    365 non-null float64
TMIN    365 non-null float64
WDF2    365 non-null float64
WDF5    353 non-null float64
WSF2    365 non-null float64
WSF5    353 non-null float64
WT01    120 non-null float64
WT02    15 non-null float64
WT03    1 non-null float64
WT08    73 non-null float64
dtypes: float64(15), object(1)
memory usage: 44.2+ KB


In [113]:
years= pd.concat([y1, y2, y3])

In [114]:
years.date.head()

0    2014-05-03T00:00:00
1    2014-05-04T00:00:00
2    2014-05-05T00:00:00
3    2014-05-06T00:00:00
4    2014-05-07T00:00:00
Name: date, dtype: object

In [115]:
years.date.tail()

358    2017-04-26T00:00:00
359    2017-04-27T00:00:00
360    2017-04-28T00:00:00
361    2017-04-29T00:00:00
362    2017-04-30T00:00:00
Name: date, dtype: object

In [116]:
years.to_csv('LAX_3years.csv', index= False)